## [인과관계추론] 김장현 (2019-26471)

In [1]:

!pip install transformers  #if colab !!!!!!!!
cur_dir="./drive/MyDrive/NLP/final/"   #if colab !!!!!!!!

#cur_dir="./" # if local!!!!!!!!!!!!!!1

In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import transformers
import pandas
from transformers import AutoTokenizer
from transformers import RobertaModel, RobertaConfig
from transformers import AdamW
import time
import argparse
from tqdm.auto import tqdm
import numpy as np
import random

transformers.logging.set_verbosity(40) # Turn off warning

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:


save_dir = cur_dir+'result_wic'


In [5]:

seed=777

torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)



### Load QA data
- Train, Dev 데이터가 base_path에 들어있어야 합니다. (default: './data')

In [6]:
"""base_path = cur_dir+'data'

def load_data(path, tokenizer):
    dataset = pandas.read_csv(path,
                              delimiter='\t',
                              names=['ID', 'Target', 'text1', 'text2','answer','s_s1','e_s1','s_s2','e_s2'],
                              header=0)

    dataset_text1=[]
    dataset_text2=[]

    for i,text1 in enumerate(dataset['text1']):
        text= text1[:dataset["s_s1"][i]]+"[WORD1S]"+text1[dataset["s_s1"][i]:dataset["e_s1"][i]]+"[WORD1E]"+text1[dataset["e_s1"][i]:]
        dataset_text1.append(text)

    for i,text2 in enumerate(dataset['text2']):
        text= text2[:dataset["s_s2"][i]]+"[WORD2S]"+text2[dataset["s_s2"][i]:dataset["e_s2"][i]]+"[WORD2E]"+text2[dataset["e_s2"][i]:]
        dataset_text2.append(text)
    
    word1s_tok_idx=tokenizer.encode("[WORD1S]")[1]
    word1e_tok_idx=tokenizer.encode("[WORD1E]")[1]
    word2s_tok_idx=tokenizer.encode("[WORD2S]")[1]
    word2e_tok_idx=tokenizer.encode("[WORD2E]")[1]

    tokenized = tokenizer(dataset_text1,
                          dataset_text2,
                          padding=True,
                          truncation=True,
                          return_tensors="pt")
    dataset['label'] = torch.tensor(dataset['answer'],dtype=int)

    #print(tokenized["input_ids"][0])

    word_mask=[torch.roll((tokenized["input_ids"]==word1e_tok_idx),-1,dims=1).int() , torch.roll((tokenized["input_ids"]==word2e_tok_idx),-1,dims=1).int() ]

    #print(word_mask[0][0])
    #print(word_mask[1][0])

    #print( torch.sum(torch.roll((tokenized["input_ids"]==worde_tok_idx),-2,dims=1).int()-(tokenized["input_ids"]==words_tok_idx).int()) )

    return dataset, tokenized, word_mask

val_dataset, val_tokenized,val_word_mask = load_data(os.path.join(base_path, 'NIKL_SKT_WiC_Dev.tsv'), tokenizer)
print(tokenizer.tokenize("그의 죽음은 타살로 [WORD1S]단정[WORD1E]이 되었다"))
print(tokenizer.encode("그의 죽음은 타살로 [WORD1S]단정[WORD1E]이 되었다"))
print(tokenizer.encode("[WORD1S]"))
print(tokenizer.encode("[WORD1E]"))
print(val_tokenized["input_ids"].shape)"""

'base_path = cur_dir+\'data\'\n\ndef load_data(path, tokenizer):\n    dataset = pandas.read_csv(path,\n                              delimiter=\'\t\',\n                              names=[\'ID\', \'Target\', \'text1\', \'text2\',\'answer\',\'s_s1\',\'e_s1\',\'s_s2\',\'e_s2\'],\n                              header=0)\n\n    dataset_text1=[]\n    dataset_text2=[]\n\n    for i,text1 in enumerate(dataset[\'text1\']):\n        text= text1[:dataset["s_s1"][i]]+"[WORD1S]"+text1[dataset["s_s1"][i]:dataset["e_s1"][i]]+"[WORD1E]"+text1[dataset["e_s1"][i]:]\n        dataset_text1.append(text)\n\n    for i,text2 in enumerate(dataset[\'text2\']):\n        text= text2[:dataset["s_s2"][i]]+"[WORD2S]"+text2[dataset["s_s2"][i]:dataset["e_s2"][i]]+"[WORD2E]"+text2[dataset["e_s2"][i]:]\n        dataset_text2.append(text)\n    \n    word1s_tok_idx=tokenizer.encode("[WORD1S]")[1]\n    word1e_tok_idx=tokenizer.encode("[WORD1E]")[1]\n    word2s_tok_idx=tokenizer.encode("[WORD2S]")[1]\n    word2e_tok_idx=to

In [7]:
def load_data(path, tokenizer):
    dataset = pandas.read_csv(path,
                              delimiter='\t',
                              names=['ID', 'Target', 'text1', 'text2','answer','s_s1','e_s1','s_s2','e_s2'],
                              header=0)

    dataset_text1=[]
    dataset_text2=[]

    for i,text1 in enumerate(dataset['text1']):
        text= text1[:dataset["s_s1"][i]]+"[WORD1S]"+text1[dataset["s_s1"][i]:dataset["e_s1"][i]]+"[WORD1E]"+text1[dataset["e_s1"][i]:]
        dataset_text1.append(text)

    for i,text2 in enumerate(dataset['text2']):
        text= text2[:dataset["s_s2"][i]]+"[WORD2S]"+text2[dataset["s_s2"][i]:dataset["e_s2"][i]]+"[WORD2E]"+text2[dataset["e_s2"][i]:]
        dataset_text2.append(text)
    
    word1s_tok_idx=tokenizer.encode("[WORD1S]")[1]
    word1e_tok_idx=tokenizer.encode("[WORD1E]")[1]
    word2s_tok_idx=tokenizer.encode("[WORD2S]")[1]
    word2e_tok_idx=tokenizer.encode("[WORD2E]")[1]

    tokenized = tokenizer(dataset_text1,
                          dataset_text2,
                          padding=True,
                          truncation=True,
                          return_tensors="pt")
    dataset['label'] = torch.tensor(dataset['answer'],dtype=int)

    #print(tokenized["input_ids"][0])

    word_mask1=  torch.roll((tokenized["input_ids"]==word1e_tok_idx),-1,dims=1).int()
    word_mask2=  torch.roll((tokenized["input_ids"]==word2e_tok_idx),-1,dims=1).int()
    #print(word_mask[0][0])
    #print(word_mask[1][0])

    #print( torch.sum(torch.roll((tokenized["input_ids"]==worde_tok_idx),-2,dims=1).int()-(tokenized["input_ids"]==words_tok_idx).int()) )

    return dataset, tokenized, word_mask1, word_mask2


class TensorDataset(Dataset):
    def __init__(self, tokenized_dataset, word_mask1, word_mask2, labels):
        self.tokenized_dataset = tokenized_dataset
        self.word_mask1= word_mask1
        self.word_mask2= word_mask2
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.tokenized_dataset.items()}
        item["word_mask1"]=self.word_mask1[idx]
        item["word_mask2"]=self.word_mask2[idx]
        label = self.labels[idx]
        return item, label

    def __len__(self):
        return len(self.labels)

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_type = "Roberta"
size = 'large'
model_name = f"klue/roberta-{size}"
tokenizer = AutoTokenizer.from_pretrained(model_name)

new_special_tokens=["[WORD1S]","[WORD1E]","[WORD2S]","[WORD2E]"]
tokenizer.add_special_tokens({"additional_special_tokens":new_special_tokens})
print(f"added {len(new_special_tokens)} new special token")

print(len(tokenizer))

added 4 new special token
32004


In [9]:

base_path = cur_dir+'data'

train_dataset, train_tokenized, train_word_mask1, train_word_mask2 = load_data(os.path.join(base_path, 'NIKL_SKT_WiC_Train.tsv'),
                                           tokenizer)
val_dataset, val_tokenized, val_word_mask1, val_word_mask2 = load_data(os.path.join(base_path, 'NIKL_SKT_WiC_Dev.tsv'), tokenizer)

train_dataset = TensorDataset(train_tokenized, train_word_mask1, train_word_mask2, train_dataset['label'])
val_dataset = TensorDataset(val_tokenized, val_word_mask1, val_word_mask2, val_dataset['label'])

#if size == 'base':
#    batch_size = 16
#else:
#    batch_size = 4
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
# 데이터 예시
tokenizer.decode(train_tokenized['input_ids'][0])

'[CLS] 그의 죽음은 타살로 [WORD1S] 단정 [WORD1E] 이 되었다. [SEP] [WORD2S] 단정 [WORD2E] 이 된 교실은 정돈되어 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

### Load pretrained model
- 모델은 KLUE-RoBERTa를 사용하였습니다. (https://github.com/KLUE-benchmark/KLUE) 
- cls token에 해당하는 hidden feature에 linear classifier를 추가했습니다.

In [11]:
a=torch.tensor([[[1,2],[3,4],[5,6]],[[111,112],[31,41],[5,6]],[[11,21],[3,4],[7,8]]])
b=torch.tensor([[1,0,0],[0,1,0],[1,0,1]])
a*b.unsqueeze(2)


tensor([[[ 1,  2],
         [ 0,  0],
         [ 0,  0]],

        [[ 0,  0],
         [31, 41],
         [ 0,  0]],

        [[11, 21],
         [ 0,  0],
         [ 7,  8]]])

In [20]:
class Roberta(RobertaModel):
    # Add classification layer to Roberta model
    def __init__(self, config, model_name, tok_len):
        super(Roberta, self).__init__(config)
        self.roberta = RobertaModel.from_pretrained(model_name, config=config)
        self.roberta.resize_token_embeddings(tok_len)
        self.hdim = config.hidden_size
        self.nclass = config.nclass
        self.classifier = nn.Linear(self.hdim*2, self.nclass)

    def forward(self, input_ids, attention_mask, word_mask1, word_mask2 , **kwargs):
        outputs = self.roberta(input_ids, attention_mask=attention_mask)[0]
        #(batch_size, sequence_length, hidden_size)

        #word_mask  ( 2, batch , seq_len)

        #word_mask[0] (batch, seq_len)  [[0,0,0,0,0,1,0,0,0,0],[...],..]
        #word_mask[0].unsqueeze(2) (batch, seq_len,1) 
        
        #torch.sum( word_mask[0].unsqueeze(2) * outputs , dim=1 )   # (batch,  hidden_size)
        #word_mask[1].unsqueeze(2) * outputs
        
        h = torch.cat( [torch.sum( word_mask1.unsqueeze(2) * outputs , dim=1 ),torch.sum( word_mask2.unsqueeze(2) * outputs , dim=1 )], dim=1)
        # (batch, hidden_size*2)

        logits = self.classifier(h)
        return logits


config = RobertaConfig.from_pretrained(model_name)
config.nclass = 2
print(config)

RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "nclass": 2,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 32000
}



In [24]:
print(type(config))

<class 'transformers.models.roberta.configuration_roberta.RobertaConfig'>


### Finetune model
- Size: **Large** (\~85%) / Base (\~79%)
- Epoch: 10
- warm up: 10% training step (없으면 불안정)
- Learning rate: 1e-5, **8e-6**, 5e-6 (큰 차이는 없으나, 커지면 불안정)
- Batch size: **5**, 20, 60
- Finetuning: **All**, Only classifier (\~57%)
- classifier: 전체를 업데이트하는 경우 multi-layer 효과 작음

In [ ]:
def train_epoch(epoch, model, train_loader, optimizer, scheduler):
    model.train()
    total_loss = 0
    cor = 0
    n_sample = 0
    s = time.time()
    criterion = nn.CrossEntropyLoss()

    for data, target in tqdm(train_loader):
        item = {key: val.to(device) for key, val in data.items()}
        target = target.to(device)

        logits = model(**item)
        loss = criterion(logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        with torch.no_grad():
            preds = torch.argmax(logits, dim=-1)

        total_loss += loss.item()
        cor += (preds == target).sum().item()
        n_sample += len(target)

        print(f"{cor}/{n_sample}", end='\r')

    loss_avg = total_loss / n_sample
    acc = cor / n_sample
    print(
        f"[Epoch {epoch}] Train loss: {loss_avg:.3f}, acc: {acc*100:.2f}, time: {time.time()-s:.1f}s"
    )
    return acc


def validate(epoch, model, val_loader, verbose=True):
    model.eval()
    total_loss = 0
    cor = 0
    n_sample = 0
    criterion = nn.CrossEntropyLoss()
    pred_all = []
    
    with torch.no_grad():
        for data, target in val_loader:
            item = {key: val.to(device) for key, val in data.items()}
            target = target.to(device)

            logits = model(**item)
            loss = criterion(logits, target)
            preds = torch.argmax(logits, dim=-1)
            pred_all.append(preds)

            total_loss += loss.item()
            cor += (preds == target).sum().item()
            n_sample += len(target)

    loss_avg = total_loss / n_sample
    acc = cor / n_sample
    pred_all = torch.cat(pred_all)
    
    if verbose:
        print(f"[Epoch {epoch}] Valid loss: {loss_avg:.3f}, acc: {acc*100:.2f}")
    return acc, pred_all


def train(idx, num_epochs, lr, train_loader, val_loader, tokenizer):
    print(f"Start trining {idx}th model")
    model = Roberta(config, model_name,len(tokenizer)).to(device)


    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = transformers.get_scheduler("linear",
                                           optimizer=optimizer,
                                           num_warmup_steps=num_epochs * len(train_loader) // 10,
                                           num_training_steps=num_epochs * len(train_loader))
    best_acc = 0
    for epoch in range(num_epochs):
        train_acc = train_epoch(epoch, model, train_loader, optimizer, scheduler)
        val_acc, _ = validate(epoch, model, val_loader)
        if val_acc > best_acc:
            best_acc = val_acc

            model_to_save = model.module if hasattr(model, "module") else model

            if not os.path.exists( os.path.join(save_dir, f'{idx}') ):
                os.makedirs( os.path.join(save_dir, f'{idx}') )

            model_to_save.save_pretrained(os.path.join(save_dir, f'{idx}'))
            
    print(f"Training finish! Best validation accuracy: {best_acc*100:.2f}\n")

In [ ]:
lr = 8e-6
num_epochs = 10

####  최적 세팅에서 10번 반복한 결과 84% \~ 87% 의 결과를 얻음
- 각 실험은 1시간 정도 소요 (6min/epoch) 
- (참고) jupyter를 서버에서 돌려서 컴퓨터 연결이 끊겼을때 print가 잘 되지 않은 경우가 있으나, 모델 학습 및 저장은 이상 없음.

In [ ]:
for i in range(10):
    train(i, num_epochs, lr, train_loader, val_loader, tokenizer)

Start trining 0th model


Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

  0%|          | 0/1937 [00:00<?, ?it/s]

[Epoch 0] Train loss: 0.128, acc: 72.24, time: 2181.8s
[Epoch 0] Valid loss: 0.089, acc: 84.82


  0%|          | 0/1937 [00:00<?, ?it/s]

...

  0%|          | 0/1937 [00:00<?, ?it/s]

[Epoch 2] Train loss: 0.021, acc: 96.99, time: 2182.9s
[Epoch 2] Valid loss: 0.080, acc: 90.74


  0%|          | 0/1937 [00:00<?, ?it/s]

[Epoch 3] Train loss: 0.010, acc: 98.72, time: 2182.3s
[Epoch 3] Valid loss: 0.099, acc: 89.71


  0%|          | 0/1937 [00:00<?, ?it/s]

### Test models (validation: 88.14%)
- 최종 모델은 위 개별 모델을 ensemble해서 얻음 
- Ensemble에서 학습 성능 하위 40% 모델들은 배제

In [ ]:
def validate_tqdm(epoch, model, val_loader, verbose=True):
    model.eval()
    total_loss = 0
    cor = 0
    n_sample = 0
    criterion = nn.CrossEntropyLoss()
    pred_all = []
    
    with torch.no_grad():
        for data, target in tqdm(val_loader):
            item = {key: val.to(device) for key, val in data.items()}
            target = target.to(device)

            logits = model(**item)
            loss = criterion(logits, target)
            preds = torch.argmax(logits, dim=-1)
            pred_all.append(preds)

            total_loss += loss.item()
            cor += (preds == target).sum().item()
            n_sample += len(target)

    loss_avg = total_loss / n_sample
    acc = cor / n_sample
    pred_all = torch.cat(pred_all)
    
    if verbose:
        print(f"[Epoch {epoch}] Valid loss: {loss_avg:.3f}, acc: {acc*100:.2f}")
    return acc, pred_all

def validate_mymodel(val_loader, answer, model_saved_name):
    model = Roberta.from_pretrained(os.path.join(save_dir, model_saved_name), model_name)
    model.to(device)
    acc, pred_all = validate_tqdm('best', model, val_loader, verbose=False)
    print(f"Load {idx}th model (acc: {acc*100:.2f})")

answer = torch.tensor(val_dataset.labels)
validate_mymodel(val_loader, answer, "0_91.51_6epoch_seed12")

TypeError: ignored

In [ ]:
def validate_ensemble(val_loader, answer, idx_max=10):
    pred_ensemble = []
    for idx in range(idx_max):
        model = Roberta.from_pretrained(os.path.join(save_dir, f'{idx}'), model_name)
        model.to(device)
        acc, pred_all = validate('best', model, val_loader, verbose=False)
        print(f"Load {idx}th model (acc: {acc*100:.2f})")
        if acc >= 0.85:
            pred_ensemble.append(pred_all)
        
    pred_ensemble = torch.stack(pred_ensemble, dim=-1).float()
    pred_ensemble = (pred_ensemble.mean(-1) >= 0.5).long().to(answer.device)
    acc_ensemble = (pred_ensemble == answer).sum() / len(answer)
    print(f"\nEnsemble accuracy: {acc_ensemble*100:.2f}")

In [ ]:
answer = torch.tensor(val_dataset.labels)

validate_ensemble(val_loader, answer, idx_max=10)